In [ ]:
from MCTSOpt import Tree
from MCTSOpt.SelectionRule_Stocastic import UBStocastic
from MCTSOpt.ParameterOpt import LogisticSearch
from time import time
import numpy as np

In [ ]:
from Model import getexploremodel
model = getexploremodel()

nparameters = 0
for layer in model.trainable_weights:
    print(layer.shape)
    nparameters += layer.shape.num_elements()
print("Number of parameters: ", nparameters)



In [ ]:
from ScoringFunction import MCTS_MacroScore
deadzonewidth = 0.25
objfunc = MCTS_MacroScore(model, deadzone=deadzonewidth)

In [ ]:
depthscale = [1.0, 0.7, 0.5, 0.3, 0.2, 0.1]
depthlimit = len(depthscale)+5

print("Dead Zone Width:",deadzonewidth)

lbounds = []
ubounds = []
for layer in model.trainable_weights:
    print(layer.shape, layer.shape.num_elements())
    if layer.ndim != 1:
        lbounds += [ 0.0-deadzonewidth]*layer.shape.num_elements()
        ubounds += [ 1.5+deadzonewidth]*layer.shape.num_elements()
    else:
        lbounds += [ -5.0-deadzonewidth]*layer.shape.num_elements()
        ubounds += [  5.0+deadzonewidth]*layer.shape.num_elements()       

startset = [0.0]*nparameters

startset = np.array(startset)
lbounds = np.array(lbounds)
ubounds = np.array(ubounds)

print("Startset size:", startset.shape)
print("Lbounds size:", lbounds.shape)
print("Ubounds size:", ubounds.shape)

#print("Startset: ", startset)
#print("Lbounds: ", lbounds, lbounds.min(), lbounds.max())
#print("Ubounds: ", ubounds, ubounds.min(), ubounds.max())


indata = LogisticSearch(parameters=startset, lossfunction=objfunc,  lbounds=lbounds, ubounds=ubounds, depthscale=depthscale)

In [ ]:




#---Tree Main Run loop---
#Critical Parameters to Set
tree = Tree(seeddata=indata, 
        playouts=20, 
        selectfunction=UBStocastic, 
        headexpansion=30,
        verbose=True
        )
tree.expand(nExpansions=1)
tree.setconstant(5.37e2)
starttime = time()
for iLoop in range(1,150):
    print("Loop Number: %s"%(iLoop))
    tree.playexpand(nExpansions=1, depthlimit=depthlimit)
    tree.autoscaleconstant(scaleboost=2.0)
    tree.simulate(nSimulations=1)
    curtime = time()
    model.save_weights("MCTSweights.h5")
    curmin = objfunc.bestscore
    print("Search Duration: %s, Best Score:%s"%(curtime-starttime, curmin))